# Imports

In [1]:
#Creating a clean Slate
from IPython import get_ipython
get_ipython().magic('reset -sf')
#Directing to the correct location and getting the file
import os
os.chdir('C:/Users/ritux/OneDrive - Danmarks Tekniske Universitet/Skrivebord/3 -Computational Tools for Data Science/')
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

C:\Users\ritux\AppData\Local\Temp\ipykernel_3476\2114608133.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


# Support Functions

In [2]:
headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
            'Accept-Language': 'en-US,en;q=0.9',
            'Referer': 'https://www.google.com/'  # Sometimes needed to bypass blocks
        }

In [3]:
def url_scrapping(base_url,limit,headers=headers):

    urls = [] # List of urls

    to_visit = [base_url] # Queue
    while to_visit and len(urls) < limit:
        
        # Getting an URL form the queue
        current_url = to_visit.pop(0)
        # Issuing the request for access to the webpage
        response = requests.get(current_url, headers=headers)
        # Ensuring we are successful in the request
        if response.status_code != 200:
            print(f"Request failed with status code {response.status_code} at url: {current_url}")
        
        # Obtaining the information in the webpage
        Soup = BeautifulSoup(response.text, 'html.parser')
        # Iterating through the linked pages from the page we are visiting
        for a in Soup.find_all('a', href=True): 
            url = urljoin(base_url, a['href'])
            # Ensuring it stays within the chosen page
            if url.startswith(base_url) and url: 
                # Keeping only new urls 
                if url not in urls:
                    urls.append(url)
                    # Adding to queue only unscheduled urls
                    if url not in to_visit: 
                        to_visit.append(url)
    return urls


In [4]:
def filter_urls(urls, limit,condition,headers=headers):
    filtered_urls = []
    for url in urls:
        try:
            # Fetch the page
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"Request failed with status code {response.status_code} at url: {url}")
            
            soup = BeautifulSoup(response.text, 'html.parser')

            # Check if the page contains the ingredient selector
            if soup.select(condition) and url not in filtered_urls:
                filtered_urls.append(url)
                
                # Stop if we've reached the limit
                if len(filtered_urls) >= limit:
                    break
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")
            continue  # Skip to the next URL in case of an error

    return filtered_urls


In [5]:
# Function to filter URLs
def bcc_filter_urls(urls, limit,headers=headers):
    filtered_urls = []
    for url in urls:
        segments = url.split('/')
        if url.startswith('https://www.bbc.co.uk/') and url.startswith('https://www.bbc.co.uk/food/recipes') and len(segments) in [6,7]:
            try:
                response = requests.get(url,headers=headers)
                if response.status_code != 200:
                    print(f"Request failed with status code {response.status_code} at url: {url}")
            
                soup = BeautifulSoup(response.text, 'html.parser')
                # Check if the page contains the ingredients
                if soup.select_one("div.recipe-ingredients-wrapper ul.recipe-ingredients__list li.recipe-ingredients__list-item a.recipe-ingredients__link")  and url not in filtered_urls:
                    filtered_urls.append(url)
                    if len(filtered_urls) >= limit:
                        break
            except requests.RequestException as e:
                print(f"Error fetching {url}: {e}")
                continue  # Skip to the next URL in case of an error

    return filtered_urls

# Body

In [6]:
bbc_url = 'https://www.bbc.co.uk/food/recipes'
all_url = "https://www.allrecipes.com"
two_url = 'https://www.twopeasandtheirpod.com/'

bbc_scraped_urls = url_scrapping(bbc_url,1000)
all_scraped_urls = url_scrapping(all_url,1000)
two_scraped_urls = url_scrapping(two_url,1000)


Request failed with status code 403 at url: https://www.twopeasandtheirpod.com/wp-login.php?redirect_to=https%3A%2F%2Fwww.twopeasandtheirpod.com%2Fsaved-recipes%2F


In [7]:
all_con = "li.mm-recipes-structured-ingredients__list-item"
two_con = ".wprm-recipe-ingredient"

bbc_url_filterd = bcc_filter_urls(bbc_scraped_urls, 1000,headers=headers)
all_url_filterd = filter_urls(all_scraped_urls, 1000,all_con,headers=headers)
two_url_filterd = filter_urls(two_scraped_urls, 1000,two_con,headers=headers)


Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/broker/facebook/login?client_id=alrcom&tab_id=PCb85YRPuiA&session_code=L-GTFLh2QNkiFK66QX2LT8xX4UGZzc0AVN8XcnS1AKs
Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/broker/google/login?client_id=alrcom&tab_id=PCb85YRPuiA&session_code=L-GTFLh2QNkiFK66QX2LT8xX4UGZzc0AVN8XcnS1AKs
Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/login-actions/registration?client_id=alrcom&tab_id=PCb85YRPuiA
Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/broker/facebook/login?client_id=alrcom&tab_id=cKMwMrHZXlg&session_code=gZmGCS6p3-j7d5hlVy6BV3gDVs4QwQkCKlISvrjD7gM
Request failed with status code 404 at url: https://www.allrecipes.com/realms/alrcom/broker/google/login?client_id=alrcom&tab_id=cKMwMrHZXlg&session_code=gZmGCS6p3-j7d5hlVy6BV3gDVs4QwQkCKlISvrjD7gM
Request failed with status code 404 at url: https://www.

C:\Users\ritux\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Request failed with status code 403 at url: https://www.twopeasandtheirpod.com/wp-login.php/


In [13]:
def two_get_ingredients(urls):
    ingredients = []
    for url in urls:
        # Fetch the page content
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        recipe_ingredients = []
        # Use soup.select with the CSS selector for ingredients
        for item in soup.select(".wprm-recipe-ingredient"):
            # Get name if they exist
            name = item.select_one(".wprm-recipe-ingredient-name")
            if name:# Add to ingredients list
                recipe_ingredients.append(name.get_text(strip=True))
                
        ingredients.append(recipe_ingredients)
    return ingredients

In [14]:
two_ingredients = two_get_ingredients(two_url_filterd)

In [15]:
def all_get_ingredients(urls):
    ingredients = []
    for url in urls:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        recipe_ingredients = []
        # Use soup.select with the CSS selector for ingredients
        for item in soup.select("ul.mm-recipes-structured-ingredients__list > li.mm-recipes-structured-ingredients__list-item"):
            # Extract the ingredient
            name = item.select_one("span[data-ingredient-name]").get_text(strip=True)
            # Combine to form a full ingredient line
            recipe_ingredients.append(name)
        ingredients.append(recipe_ingredients)
    return ingredients

In [16]:
all_ingredients = all_get_ingredients(all_url_filterd)

In [17]:
# Function to extract ingredients from BBC website
def extract_ingredients_bbc(url):
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to retrieve webpage. Status code:", response.status_code)
        return None
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    ingredient_names = []
    
    title = soup.select_one("h1.content-title__text").text.strip()
    
    ingredients_elements = soup.select("div.recipe-ingredients-wrapper ul.recipe-ingredients__list li.recipe-ingredients__list-item a.recipe-ingredients__link")  # Use CSS selector for the ingredients span elements
        
    for ingredient_element in ingredients_elements:
        ingredient_names.append(ingredient_element.text.strip())
            

    return ingredient_names


In [18]:
# Function to extract ingredients from BBC website
def bcc_get_ingredients(urls):
    ingredients = []
    for url in bbc_url_filterd:
        ingredients.append(extract_ingredients_bbc(url))
    return ingredients


In [19]:
bcc_ingredients = bcc_get_ingredients(bbc_url_filterd)

## Concatenate the three websites

In [20]:
print("For bbc:")
print("Number of URLs found :" , len(bbc_scraped_urls))
print("Number of URLs filtered:" , len(bbc_url_filterd))
print("Number of URLs with recipes:" , len(bcc_ingredients))
n_bcc =len(bcc_ingredients)
print("\n")
print("For all:")
print("Number of URLs found :" , len(all_scraped_urls))
print("Number of URLs filtered:" , len(all_url_filterd))
print("Number of URLs with recipes:" , len(all_ingredients))
n_all =len(all_ingredients)
print("\n")
print("For two:")
print("Number of URLs found :" , len(two_scraped_urls))
print("Number of URLs filtered:" , len(two_url_filterd))
print("Number of URLs with recipes:" , len(two_ingredients))
n_two =len(two_ingredients)



For bbc:
Number of URLs found : 1013
Number of URLs filtered: 552
Number of URLs with recipes: 552


For all:
Number of URLs found : 1311
Number of URLs filtered: 581
Number of URLs with recipes: 581


For two:
Number of URLs found : 1085
Number of URLs filtered: 417
Number of URLs with recipes: 417


In [21]:
count  = 0
all_ingredients_df = pd.DataFrame(columns=['id','source','link', 'title'])
for n in range(n_bcc):
    count += 1
    ingredients_df= pd.DataFrame(bcc_ingredients[n])
    ingredients_df['id'] = count
    ingredients_df['source'] = 'bbc'
    ingredients_df['link'] = bbc_url_filterd[n]
    ingredients_df['title'] = ' '    
    all_ingredients_df = pd.concat([all_ingredients_df, ingredients_df], ignore_index=True)

for n in range(n_all):
    count += 1
    ingredients_df= pd.DataFrame(all_ingredients[n])
    ingredients_df['id'] = count
    ingredients_df['source'] = 'all'
    ingredients_df['link'] = all_url_filterd[n]
    ingredients_df['title'] = ' '
    all_ingredients_df = pd.concat([all_ingredients_df, ingredients_df], ignore_index=True)

for n in range(n_two):
    count += 1
    ingredients_df= pd.DataFrame(two_ingredients[n])
    ingredients_df['id'] = count
    ingredients_df['source'] = 'two'
    ingredients_df['link'] = two_url_filterd[n]
    ingredients_df['title'] = ' '
    all_ingredients_df = pd.concat([all_ingredients_df, ingredients_df], ignore_index=True)


In [22]:
all_ingredients_df

,id,source,link,title,0
0,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,beef mince
1,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,red onion
2,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,garlic
3,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,courgette
4,1,bbc,https://www.bbc.co.uk/food/recipes/healthy_chi...,,aubergine
...,...,...,...,...,...
18668,1550,two,https://www.twopeasandtheirpod.com/pumpkin-bun...,,100% pure pumpkin
18669,1550,two,https://www.twopeasandtheirpod.com/pumpkin-bun...,,"cream cheese,"
18670,1550,two,https://www.twopeasandtheirpod.com/pumpkin-bun...,,confectioner's sugar
18671,1550,two,https://www.twopeasandtheirpod.com/pumpkin-bun...,,vanilla extract


In [23]:
all_ingredients_df.to_csv('recipes_data.csv', index=False)

# Saving

In [25]:
scraped_urls = bbc_scraped_urls + all_scraped_urls + two_scraped_urls
scraped_urls_df = pd.DataFrame({'scraped_urls': scraped_urls})
scraped_urls_df.to_csv('scraped_urls.csv', index=False)

In [26]:
url_filterd = bbc_url_filterd + all_url_filterd + two_url_filterd
url_filterd_df = pd.DataFrame({'url_filterd': url_filterd})
url_filterd_df.to_csv('url_filterd.csv', index=False)

# Data

In [27]:
from nltk.corpus import stopwords
from nltk import FreqDist, bigrams
from nltk.tokenize import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud

In [28]:
df = pd.read_csv('recipes_data.csv')

In [29]:
# Median number of ingredients per id
median_ingredients = df.groupby('id').size().median()
print(f"Average: {median_ingredients}")

# Minimum number of ingredients per id
min_ingredients = df.groupby('id').size().min()
print(f"Minimum: {min_ingredients}")

# Maximum number of ingredients per id
max_ingredients = df.groupby('id').size().max()
print(f"Maximum: {max_ingredients}")

# Standard deviation of the number of ingredients per id
std_ingredients = df.groupby('id').size().std()
print(f"Standard deviation: {std_ingredients}")

Average: 12.0
Minimum: 1
Maximum: 44
Standard deviation: 5.382200654329305


In [31]:
words = [word for document in df['0'] for word in str(document).split()]

# Word Count
total_words = len(words)
print(f'Total words: {total_words}')

# Unique Word Count
unique_words = len(set(words))
print(f'Unique words: {unique_words}')



# Most Common Words (excluding stopwords)
stop_words = set(stopwords.words('english'))
non_stopwords = [word for word in words if word not in stop_words]
most_common_words = Counter(non_stopwords).most_common(15)
print(f'Most common words: {most_common_words}')

# Most Common Stop Words
stopwords_in_text = [word for word in words if word in stop_words]
most_common_stopwords = Counter(stopwords_in_text).most_common(15)
print(f'Most common stop words: {most_common_stopwords}')
#count of stop words in df
print(f"Total amount of stop words: {len(stopwords_in_text)}")


Total words: 40594
Unique words: 2193
Most common words: [('salt', 1065), ('oil', 901), ('pepper', 878), ('ground', 837), ('sugar', 806), ('black', 723), ('butter', 569), ('flour', 569), ('chopped', 516), ('olive', 494), ('garlic', 488), ('chicken', 455), ('powder', 405), ('vanilla', 395), ('cream', 394)]
Most common stop words: [('or', 502), ('and', 475), ('to', 347), ('of', 192), ('into', 139), ('as', 116), ('for', 92), ('more', 82), ('can', 37), ('with', 33), ('in', 25), ('if', 8), ('at', 6), ('such', 6), ('on', 6)]
Total amount of stop words: 2102
